In [1]:
import numpy as np
from scipy.interpolate import BSpline
import torch
from torchvision import models, transforms
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense, Flatten, Dropout, Concatenate, Input, Conv2D, MaxPooling2D, LSTM, BatchNormalization
from tensorflow.keras.models import Model
import sympy as sp
from sympy import symbols, diag, Matrix, BlockMatrix
import cv2
import pytesseract
import ast
import textwrap


In [2]:
class FilePreprocessor:
    def __init__(self, path_to_file):
        self.path_to_file = path_to_file

        # List of rules/action key pair.
        # Add your new rule and how to process the text (function) here
        self.rules = [(self._is_python_file, self._process_if_python)]

    def process_text(self, text: str) -> str:
        """
        Process the text based on the internal rules.
        """
        for condition, action in self.rules:
            if condition():
                return action(text)
        return text  # Return the text unchanged if no rules apply

    
    def preprocess_text_file(file_path):
      with open(file_path, 'r') as file:
        content = file.read()
        preprocessed_content = [ord(char) for char in content]
        tensor_content = tf.convert_to_tensor(preprocessed_content, dtype=tf.float32)
        tensor_content = tf.expand_dims(tensor_content, 0)
        return tensor_content

    def preprocess_audio_file(file_path):
        audio, _ = librosa.load(file_path, sr=16000)
        audio = librosa.resample(audio, orig_sr=16000, target_sr=8000)
        tensor_audio = tf.convert_to_tensor(audio, dtype=tf.float32)
        tensor_audio = tf.expand_dims(tensor_audio, 0)
        tensor_audio = tf.expand_dims(tensor_audio, -1)
        return tensor_audio

    def preprocess_video_file(file_path):
        video = []
        for _ in range(10):
            frame = tf.random.uniform((224, 224, 3))
            video.append(frame)
        tensor_video = tf.stack(video)
        tensor_video = tf.expand_dims(tensor_video, 0)
        return tensor_video
        
    def _is_python_file(self) -> bool:
        """
        Rule to check if the file is a Python file.
        """
        return self.path_to_file.endswith(".py")

    def _process_if_python(self, text: str) -> str:
        """
        Action to process Python files by checking for class definitions and indenting if found.
        """
        if self._contains_class_definition():
            return textwrap.indent(text, "    ")
        return text

    

    def _contains_class_definition(self) -> bool:
        """
        Check if the file contains a Python class definition using the ast module.
        """
        try:
            with open(self.path_to_file, "r") as file:
                content = file.read()
            parsed_ast = ast.parse(content)
            for node in ast.walk(parsed_ast):
                if isinstance(node, ast.ClassDef):
                    return True
        except SyntaxError as e:
            print(f"Syntax error when parsing the file: {e}")
        return False


class PositionalEncodingLayer(tf.keras.layers.Layer):
    def __init__(self, max_len, embed_dim, **kwargs):
        super(PositionalEncodingLayer, self).__init__(**kwargs)
        self.pos_encoding = self.positional_encoding(max_len, embed_dim)

    def positional_encoding(self, max_len, embed_dim):
        pos_enc = np.zeros((max_len, embed_dim))
        for pos in range(max_len):
            for i in range(0, embed_dim, 2):
                pos_enc[pos, i] = np.sin(pos / (10000 ** ((2 * i) / embed_dim)))
                if i + 1 < embed_dim:
                    pos_enc[pos, i + 1] = np.cos(pos / (10000 ** ((2 * (i + 1)) / embed_dim)))
        return tf.cast(pos_enc, dtype=tf.float32)

    def call(self, inputs):
        seq_len = tf.shape(inputs)[1]
        return inputs + self.pos_encoding[:seq_len, :]

class CustomEmbeddingLayer(Layer):
    def __init__(self, embed_dim):
        super(CustomEmbeddingLayer, self).__init__()
        self.embed_dim = embed_dim
        self.text_embedding = Dense(embed_dim)
        self.image_conv = tf.keras.Sequential([
            Conv2D(32, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Flatten(),
            Dense(embed_dim, activation='relu')
        ])
        self.audio_conv = tf.keras.Sequential([
            Conv2D(32, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Flatten(),
            Dense(embed_dim, activation='relu')
        ])
        self.video_conv = tf.keras.Sequential([
            Conv2D(32, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Flatten(),
            Dense(embed_dim, activation='relu')
        ])

    def build(self, input_shape):
        text_shape, image_shape, audio_shape, video_shape = input_shape
        self.text_embedding.build((text_shape[0], text_shape[1]))  
        self.built = True

    def call(self, inputs):
        text_input, image_input, audio_input, video_input = inputs

        text_embed = self.text_embedding(text_input)
        image_embed = self.image_conv(image_input)
        audio_embed = self.audio_conv(audio_input)
        video_embed = self.video_conv(video_input)

        return Concatenate()([text_embed, image_embed, audio_embed, video_embed])

    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], self.embed_dim * 4)

class MultiHeadSelfAttention(Layer):
    def __init__(self, embed_dim, num_heads):
        super(MultiHeadSelfAttention, self).__init__()
        self.num_heads = num_heads
        self.embed_dim = embed_dim
        self.depth = embed_dim // num_heads

        self.wq = Dense(embed_dim)
        self.wk = Dense(embed_dim)
        self.wv = Dense(embed_dim)
        self.dense = Dense(embed_dim)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q, mask=None):
        batch_size = tf.shape(q)[0]
        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)

        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        matmul_qk = tf.matmul(q, k, transpose_b=True)
        depth = tf.cast(tf.shape(k)[-1], tf.float32)
        logits = matmul_qk / tf.math.sqrt(depth)

        if mask is not None:
            logits += (mask * -1e9)

        attention_weights = tf.nn.softmax(logits, axis=-1)
        output = tf.matmul(attention_weights, v)
        output = tf.transpose(output, perm=[0, 2, 1, 3])
        output = tf.reshape(output, (batch_size, -1, self.embed_dim))
        output = Dropout(0.1)(output)
        return self.dense(output)

class TransformerEncoderLayer(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerEncoderLayer, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(embed_dim)
        ])
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, num_layers, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.encoder_layers = [TransformerEncoderLayer(embed_dim, num_heads, ff_dim, rate) for _ in range(num_layers)]

    def call(self, inputs, training=False):
        x = inputs
        for encoder_layer in self.encoder_layers:
            x = encoder_layer(x, training=training)
        return x


class HyperMatrix(Layer):
    def __init__(self):
        self.x = symbols('x1:10')  
        self.metric_3d = diag(1, 1, 1)
        self.metric_6d = diag(1, 1, 1, 1, 1, 1)
        # Adjusting the blocks to ensure same dimensions for block matrix
        self.block_3d = Matrix(3, 3, lambda i, j: self.metric_3d[i, j] if i == j else 0)
        self.block_6d = Matrix(3, 3, lambda i, j: self.metric_3d[i % 3, j % 3] if i % 3 == j % 3 else 0)  # Adjusting to fit 3x3
        self.hyper_block = BlockMatrix([[self.block_3d, self.block_3d], [self.block_3d, self.block_3d]])

    def get_hyper_matrix(self):
        return self.hyper_block.as_explicit()


class HyperGraph(Layer):
    def __init__(self, num_nodes):
        self.num_nodes = num_nodes
        self.nodes = symbols(f'n1:{num_nodes+1}')
        self.adjacency_matrix = Matrix(np.random.randint(2, size=(num_nodes, num_nodes)))

    def get_adjacency_matrix(self):
        return self.adjacency_matrix


class SynchronicityMatrix:
    def __init__(self, dimensions):
        self.dimensions = dimensions
        self.matrix = np.random.rand(dimensions, dimensions)

    def get_synchronicity_matrix(self):
        return Matrix(self.matrix)


class QuantumLSTMCell(tf.keras.layers.Layer):
    def __init__(self, units):
        super(QuantumLSTMCell, self).__init__()
        self.units = units
        self.lstm_cell = LSTM(units, return_sequences=True, return_state=True)

    def call(self, inputs, states):
        output, state_h, state_c = self.lstm_cell(inputs, initial_state=states)
        return output, [state_h, state_c]

    def get_initial_state(self, inputs):
        return self.lstm_cell.get_initial_state(inputs)


class TransformerModel(tf.keras.Model):
    def __init__(self, embed_dim, num_heads, ff_dim, num_layers, max_len, rate=0.1):
        super(TransformerModel, self).__init__()
        self.embedding = CustomEmbeddingLayer(embed_dim)
        self.pos_encoding = PositionalEncodingLayer(max_len, embed_dim)
        self.transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim, num_layers, rate)
        self.dense_output = Dense(embed_dim)
        self.dropout = Dropout(rate)
        self.hyper_matrix = HyperMatrix()
        self.lstm = QuantumLSTMCell
        self.graph = HyperGraph
        self.r = SynchronicityMatrix

    def call(self, inputs, training=False):
        embedded = self.embedding(inputs)
        pos_encoded = self.pos_encoding(embedded)
        transformed = self.transformer_block(pos_encoded, training=training)
        output = self.dense_output(transformed)
        output = self.dropout(output, training=training)
        return output

    def build_model(self, input_shapes):
        text_input = Input(shape=(input_shapes[0][1],), name='text_input')
        image_input = Input(shape=(input_shapes[1][0], input_shapes[1][1], input_shapes[1][2]), name='image_input')
        audio_input = Input(shape=(input_shapes[2][0], input_shapes[2][1]), name='audio_input')
        video_input = Input(shape=(input_shapes[3][0], input_shapes[3][1], input_shapes[3][2]), name='video_input')

        text_flatten = Flatten()(text_input)
        text_dense = Dense(256, activation='relu')(text_flatten)

        image_conv = Conv2D(32, (3, 3), activation='relu')(image_input)
        image_pool = MaxPooling2D((2, 2))(image_conv)
        image_flatten = Flatten()(image_pool)
        image_dense = Dense(256, activation='relu')(image_flatten)

        audio_flatten = Flatten()(audio_input)
        audio_dense = Dense(256, activation='relu')(audio_flatten)

        video_conv = Conv2D(32, (3, 3), activation='relu')(video_input)
        video_pool = MaxPooling2D((2, 2))(video_conv)
        video_flatten = Flatten()(video_pool)
        video_dense = Dense(256, activation='relu')(video_flatten)

        combined_features = Concatenate()([text_dense, image_dense, audio_dense, video_dense])
        transformer_output = self.call(combined_features)
        output = Dense(10, activation='softmax')(transformer_output)

        model = Model(inputs=[text_input, image_input, audio_input, video_input], outputs=output)
        return model




# Parameters
embed_dim = 512
num_heads = 8
ff_dim = 2048
num_layers = 6
max_len = 1000

# Instantiate the model
# Instantiate the model
transformer_model = TransformerModel(embed_dim, num_heads, ff_dim, num_layers, max_len)

# Define the input shapes
input_shapes = [(None, 100), (1920, 1080, 3), (1000, 1), (225, 225, 3)]

# Build the model
model = transformer_model.build_model(input_shapes)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


# Example paths
file_path = 'C:/Users/jonny/Documents/PATH/ONI/knowledge_base/remembered_texts/(Law, state, and society series) Karl Marx - Marx and Engels on Law-Academic Pr (1979).pdf'

# Preprocess file and convert to tensor
file_tensor = preprocess_text_file(file_path)

# Dummy tensors for other inputs
processed_input = tf.zeros((1, 1920, 1080, 3), dtype=tf.float32)
image_input = tf.zeros((1, 1920, 1080, 3), dtype=tf.float32)
audio_input = tf.zeros((1, 1000, 1), dtype=tf.float32)
video_input = tf.zeros((1, 225, 225, 3), dtype=tf.float32)

text_file_path = 'path/to/text/file.txt'
image_file_path = 'path/to/image/file.jpg'
audio_file_path = 'path/to/audio/file.wav'
video_file_path = 'path/to/video/file.mp4'

# Preprocess the inputs
text_tensor = preprocess_text_file('/PATH/ONI/knowledge_base/remembered_texts/')
image_tensor = preprocess_image_file('PATH/ONI/knowledge_base/rembered_images/')
audio_tensor = preprocess_audio_file('PATH/ONI/ltm_path/heard/')
video_tensor = preprocess_video_file('PATH/ONI/ltm_path/episodes/')

# Example batch input
inputs = [text_tensor, image_tensor, audio_tensor, video_tensor]

# Dummy target
target = np.array([[1]]) 

# Define a dummy model for illustration
input_layers = [tf.keras.layers.Input(shape=(1920, 1080, 3)),
                tf.keras.layers.Input(shape=(1920, 1080, 3)),
                tf.keras.layers.Input(shape=(1000, 1)),
                tf.keras.layers.Input(shape=(225, 225, 3)),
                tf.keras.layers.Input(shape=(None,))]

# Combine inputs (for example purposes, just concatenate them)
combined = tf.keras.layers.Concatenate()(input_layers)

# Define the model (simple example)
model = tf.keras.Model(inputs=input_layers, outputs=combined)
model.compile(optimizer='adam', loss='mse')

# Train the model
model.train_on_batch(inputs, processed_input)

# Create an instance of HyperMatrix
hyper_matrix = HyperMatrix()

# Simulate dynamic learning using HyperMatrix
input_data = inputs.reshape(1, 11, 9)

# Reshape output_data to have shape (1, 9)
output_data = outputs.reshape(1, 9)

model.summary()

# Simulate dynamic learning using HyperMatrix
hyper_matrix.simulate_dynamic_learning(input_data, output_data, epochs=5)

ValueError: not enough values to unpack (expected 4, got 2)

In [29]:

# Define a simple B-Spline function
def b_spline(x, knots, coeffs, degree=3):
    k = degree
    t = np.concatenate(([knots[0]] * k, knots, [knots[-1]] * k))
    c = coeffs
    spline = BSpline(t, c, k)
    return spline(x)

# KAN Layer
class KANLayer:
    def __init__(self, input_dim, output_dim, spline_degree=3):
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.spline_degree = spline_degree
        
        # Initialize B-spline parameters
        self.knots = [np.linspace(0, 1, 10) for _ in range(output_dim)]
        self.coeffs = [np.random.randn(10 + spline_degree - 1) for _ in range(output_dim)]

    def forward(self, x):
        out = np.zeros((x.shape[0], self.output_dim))
        for j in range(self.output_dim):
            sum_b_spline = np.sum([b_spline(x[:, i], self.knots[j], self.coeffs[j], self.spline_degree) for i in range(self.input_dim)], axis=0)
            out[:, j] = sum_b_spline
        return out

# KAN Model
class KANModel:
    def __init__(self, input_dim, hidden_dim, output_dim, spline_degree=3):
        self.layer1 = KANLayer(input_dim, hidden_dim, spline_degree)
        self.layer2 = KANLayer(hidden_dim, output_dim, spline_degree)

    def forward(self, x):
        out = self.layer1.forward(x)
        out = self.layer2.forward(out)
        return out

    # Loss function: Mean Squared Error (MSE)
    def mse_loss(y_pred, y_true):
        return np.mean((y_pred - y_true) ** 2)
    
    # Training function
    def train_kan(model, x_train, y_train, epochs=1000, lr=0.001):
        for epoch in range(epochs):
            # Forward pass
            y_pred = model.forward(x_train)
            
            # Compute loss
            loss = mse_loss(y_pred, y_train)
            
            # Compute gradients (this is a placeholder; normally you would use backpropagation)
            grad = 2 * (y_pred - y_train) / y_train.size
            
            # Update B-spline coefficients
            for layer in [model.layer1, model.layer2]:
                for j in range(layer.output_dim):
                    for i in range(layer.input_dim):
                        # Update each coefficient
                        layer.coeffs[j] -= lr * grad[:, j].dot(b_spline(x_train[:, i], layer.knots[j], layer.coeffs[j], layer.spline_degree))
    
            # Print loss every 100 epochs
            if epoch % 100 == 0:
                print(f'Epoch {epoch}, Loss: {loss:.4f}')


In [ ]:
#training loop

In [30]:

# Example usage with multi-modal input
if __name__ == "__main__":
    # Create a KAN model
    input_dim = 768  # Example for BERT embeddings
    hidden_dim = 512
    output_dim = 1
    model = KANModel(input_dim, hidden_dim, output_dim)

    # Text Processing Example
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    bert_model = BertModel.from_pretrained('bert-base-uncased')

    # Example text
    texts = ["Hello, how are you?", "I am fine, thank you!"]
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    text_embeddings = outputs.last_hidden_state.mean(dim=1).numpy()

    # Train the model on text embeddings
    y_train = np.array([[1], [0]])  # Example target
    train_kan(model, text_embeddings, y_train)

    # Video Processing Example
    # Assuming a function extract_video_frames that extracts frames and returns a list of PIL images
    video_frames = extract_video_frames("path_to_video.mp4")
    preprocess = transforms.Compose([cnn_model = models.resnet18(pretrained=True)])
    cnn_model.eval()
    video_embeddings = []
    for frame in video_frames:
        input_tensor = preprocess(frame)
        input_batch = input_tensor.unsqueeze(0)
        with torch.no_grad():
            output = cnn_model(input_batch)
        video_embeddings.append(output.numpy())
    video_embeddings = np.mean(video_embeddings, axis=0)

    # Combine text and video embeddings
    combined_embeddings = np.concatenate((text_embeddings, video_embeddings), axis=1)

    # Train the model on combined embeddings
    y_train = np.array([[1], [0]])  # Example target
    train_kan(model, combined_embeddings, y_train)

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (110976669.py, line 27)

In [7]:
import numpy as np
import tensorflow as tf
import tkinter as tk
from tkinter import filedialog
import cv2
import os

# Placeholder functions for your custom model's text processing


def generate_response(model, text_input, image_input=None, audio_input=None, video_input=None):
    # Ensure that we provide four inputs to the model
    if image_input is None:
        image_input = np.zeros((1, 1920, 1080, 3), dtype=np.float32)  # Dummy image input
    if audio_input is None:
        audio_input = np.zeros((1, 1000, 1), dtype=np.float32)        # Dummy audio input
    if video_input is None:
        video_input = np.zeros((1, 225, 225, 3), dtype=np.float32)    # Dummy video input
    
    inputs = [text_input, image_input, audio_input, video_input]
    response = model.predict(inputs)  # Adjust this line based on your model's prediction method
    return response

# Function to handle video capture
def capture_video_frame():
    cap = cv2.VideoCapture(0)
    ret, frame = cap.read()
    cap.release()
    if ret:
        return frame
    else:
        return np.zeros((225, 225, 3), dtype=np.float32)  # Dummy video input

# Initialize the Tkinter root
root = tk.Tk()
root.title("Chatbot")

# Create the conversation display
conversation_frame = tk.Frame(root)
conversation_frame.pack(pady=10)

conversation_list = tk.Listbox(conversation_frame, width=80, height=20)
conversation_list.pack(side=tk.LEFT, fill=tk.BOTH)

scrollbar = tk.Scrollbar(conversation_frame, orient=tk.VERTICAL)
scrollbar.config(command=conversation_list.yview)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

conversation_list.config(yscrollcommand=scrollbar.set)

# Function to handle user input and generate response
def send_message(event=None):
    user_input = user_entry.get()
    if user_input:
        conversation_list.insert(tk.END, f"You: {user_input}")
        user_entry.delete(0, tk.END)
        
        # Process the input and generate response using the custom model
        processed_input = process_input_text(user_input)
        
        # Capture video frame
        video_frame = capture_video_frame()
        
        response_text = generate_response(trans_model, processed_input, video_input=video_frame)
        
        conversation_list.insert(tk.END, f"Bot: {response_text}")
        conversation_list.yview(tk.END)  # Scroll to the bottom

# Create the entry widget for user input
user_entry = tk.Entry(root, width=80)
user_entry.pack(pady=10)
user_entry.bind("<Return>", send_message)

# Create the send button
send_button = tk.Button(root, text="Send", command=send_message)
send_button.pack()

# Function to handle file upload
def upload_file():
    file_path = filedialog.askopenfilename()
    if file_path:
        conversation_list.insert(tk.END, f"File uploaded: {file_path}")
        # You can add further processing of the uploaded file here

# Create the file upload button
upload_button = tk.Button(root, text="Upload File", command=upload_file)
upload_button.pack()

# Function to handle microphone input
def record_audio():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        conversation_list.insert(tk.END, "Listening...")
        audio_data = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(audio_data)
            conversation_list.insert(tk.END, f"You (via microphone): {text}")
            user_entry.delete(0, tk.END)
            
            # Process the input and generate response using the custom model
            processed_input = process_input_text(text)
            
            # Capture video frame
            video_frame = capture_video_frame()
            
            response_text = generate_response(trans_model, processed_input, video_input=video_frame)
            
            conversation_list.insert(tk.END, f"Bot: {response_text}")
            conversation_list.yview(tk.END)  # Scroll to the bottom
        except sr.UnknownValueError:
            conversation_list.insert(tk.END, "Could not understand the audio")
        except sr.RequestError as e:
            conversation_list.insert(tk.END, f"Error with speech recognition service; {e}")

# Create the microphone button
microphone_button = tk.Button(root, text="Microphone", command=record_audio)
microphone_button.pack()

# Training loop to use files from ./knowledge_base
def training_loop(model, knowledge_base_dir):
    for filename in os.listdir(knowledge_base_dir):
        file_path = os.path.join(knowledge_base_dir, filename)
        if os.path.isfile(file_path):
            try:
               with open(file_path, 'r', encoding='utf-8') as file:
                    data = file.read()
                    processed_input = process_input_text(data)
                    # We will use dummy values for other inputs during training
            except UnicodeDecodeError:
                print(f"Error decoding file: {file_path}")
                processed_input = process_input_text(file_path)
                # We will use dummy values for other inputs during training
                image_input = np.zeros((1, 1920, 1080, 3), dtype=np.float32)
                audio_input = np.zeros((1, 1000, 1), dtype=np.float32)
                video_input = np.zeros((1, 225, 225, 3), dtype=np.float32)
                            
                # Train the model
                model.train_on_batch([processed_input, image_input, audio_input, video_input], processed_input)  # Adjust this line based on your model's training method

# Path to the knowledge base directory
knowledge_base_dir = 'C:/Users/jonny/Documents/PATH/ONI/knowledge_base/remembered_texts/'
# Start the training loop
training_loop(trans_model, knowledge_base_dir)

# Run the Tkinter main loop
root.mainloop()

Error decoding file: C:/Users/jonny/Documents/PATH/ONI/knowledge_base/remembered_texts/(Law, state, and society series) Karl Marx - Marx and Engels on Law-Academic Pr (1979).pdf


ValueError: In a nested call() argument, you cannot mix tensors and non-tensors. Received invalid mixed argument: inputs=['C:/Users/jonny/Documents/PATH/ONI/knowledge_base/remembered_texts/(Law, state, and society series) Karl Marx - Marx and Engels on Law-Academic Pr (1979).pdf', <tf.Tensor 'data:0' shape=(1, 1920, 1080, 3) dtype=float32>, <tf.Tensor 'data_1:0' shape=(1, 1000, 1) dtype=float32>, <tf.Tensor 'data_2:0' shape=(1, 225, 225, 3) dtype=float32>]

In [37]:
import os
import json
import sqlite3
import numpy as np
import PyPDF2
from tensorflow.keras.models import load_model

class Memory:
    def __init__(self, working_memory_capacity=5, stm=[], ltm=[], ltm_capacity=1000000000000000000000, corpus=[], episodic_memory=[], semantic_memory=[]):
        self.working_memory_capacity = working_memory_capacity
        self.ltm_capacity = ltm_capacity
        self.stm = stm
        self.ltm = ltm
        self.episodic_memory = episodic_memory
        self.semantic_memory = semantic_memory
        self.corpus = corpus
        self.ltm_path = os.path.join('Downloads/ONI/ltm_path/', "ltm_data.json")
        self.models = {}

        # Initialize the Hopfield network
        self.hopfield_network = self.initialize_hopfield_network()

        self.load_long_term_memory(ltm)

    def handle_media(self, data):
        media_extensions = {
            '.mov': 'video',
            '.mp4': 'video',
            '.avi': 'video',
            '.wav': 'audio',
            '.mp3': 'audio',
            '.txt': 'plaintext',
            '.pdf': 'PDF',
            '.doc': 'Document File',
            'ODT': 'Open Document TXT',
            '.py': 'python file',
            '.html': 'website',
            '.js': 'javascript',
            '.css': 'styles'
        }
        file_extension = os.path.splitext(data)[1]
        media_type = media_extensions.get(file_extension, 'unknown')

        if media_type in ['video', 'audio']:
            return {'type': media_type, 'path': data}
        else:
            return None

    def load_long_term_memory(self, ltm):
        if os.path.exists(self.ltm_path):
            with open(self.ltm_path, 'r') as file:
                loaded_data = json.load(file)
                self.ltm = loaded_data
                for tokens in loaded_data.get('ltm', []):
                    if tokens not in self.semantic_memory:
                        self.semantic_memory.append(tokens)

    @classmethod
    def update_episodic_memory(cls, data, data_type, key):
        if not hasattr(cls, 'episodic_memory'):
            cls.episodic_memory = {}
        cls.episodic_memory[key] = {'data': data, 'type': data_type}

    @classmethod
    def retrieve_from_episodic(cls, key):
        if not hasattr(cls, 'episodic_memory'):
            cls.episodic_memory = {}
        return cls.episodic_memory.get(key, {}).get('data', None)

    def update_semantic_memory(self, text, media_type=None):
        if media_type:
            if media_type not in ['.txt', '.pdf', '.doc', '.py', '.js', '.css', '.html']:
                raise ValueError("Unsupported media type")

        tokens = text.split()
        for token in tokens:
            if token not in self.semantic_memory:
                self.semantic_memory.append(token)

        if len(self.semantic_memory) > self.ltm_capacity:
            raise MemoryError("Semantic memory has exceeded its capacity")

    def lookup_token(self, token):
        return self.semantic_memory.index(token) if token in self.semantic_memory else -1

    def meditate(self):
        unique_data = set(self.semantic_memory + self.working_memory)
        self.ltm = list(unique_data)[:self.ltm_capacity]

    def dream(self, model_name='default_model.h5'):
        if model_name in self.models:
            model = self.models[model_name]
        else:
            model = load_model(model_name)
            self.models[model_name] = model

        for memory in self.episodic_memory:
            data = self.handle_media(memory['path'])
            if data:
                processed_data = model.predict(data)
                memory['processed'] = processed_data

    def sleep(self, model_name='default_model.h5'):
        print("AI is going to sleep...")
        self.meditate()
        self.dream(model_name=model_name)
        self.save_long_term_memory()
        print("AI has woken up with refreshed memories.")

    def save_long_term_memory(self):
        with open(self.ltm_path, 'w') as file:
            json.dump(self.ltm, file)

    def handle_pdf(self, pdf_path):
        try:
            with open(pdf_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                num_pages = len(pdf_reader.pages)
                for page_num in range(num_pages):
                    page = pdf_reader.pages[page_num]
                    text = page.extract_text()
                    self.update_semantic_memory(text)

            print(f"PDF '{os.path.basename(pdf_path)}' added to semantic memory.")
        except Exception as e:
            print(f"Error processing PDF: {e}")

    def update_memory(self, stm_data, episodic_data=None, semantic_data=None):
        self.stm.append(stm_data)
        if episodic_data:
            self.update_episodic_memory(episodic_data)
        if semantic_data:
            self.update_semantic_memory(semantic_data)
        self.save_long_term_memory()

    def categorize_and_store(self, db_path='personality.db'):
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS personalities (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT,
                race TEXT,
                origin TEXT,
                age INTEGER,
                type TEXT
            )
        """)
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS conversations (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                personality_id INTEGER,
                timestamp TEXT,
                input_text TEXT,
                response_text TEXT,
                FOREIGN KEY (personality_id) REFERENCES personalities(id)
            )
        """)
        conn.commit()
        conn.close()

    def initialize_hopfield_network(self):
        # Initialize the Hopfield network with zeros
        self.hopfield_weights = np.zeros((self.ltm_capacity, self.ltm_capacity))

    def train_hopfield_network(self, patterns):
        for p in patterns:
            self.hopfield_weights += np.outer(p, p)
        np.fill_diagonal(self.hopfield_weights, 0)

    def recall_hopfield_network(self, pattern, steps=5):
        for _ in range(steps):
            pattern = np.sign(self.hopfield_weights @ pattern)
        return pattern

# Example usage:
mem = Memory()
mem.update_memory("New STM data")
mem.update_memory(episodic_data="example.mp3")
mem.update_semantic_memory('Documents/PATH/ONI/IdentityPersonalANDSocialinpressreformatted.pdf', media_type='.pdf')
mem.save_long_term_memory()
mem.load_long_term_memory(mem.ltm)

print("Memory system upgraded with episodic and semantic capabilities.")
print(mem.ltm)


NameError: name 'json' is not defined

In [ ]:
import numpy as np
import tensorflow as tf
import tkinter as tk
from tkinter import filedialog

import cv2
import os

# Placeholder functions for your custom model's text processing
def process_input_text(input_text):
    # Implement this to convert input_text to the format your model expects
    # For example, if the model expects tokenized input, tokenize the text here
    return input_text

def generate_response(model, text_input, image_input=None, audio_input=None, video_input=None):
    # Ensure that we provide four inputs to the model
    if image_input is None:
        image_input = np.zeros((1, 1920, 1080, 3), dtype=np.float32)  # Dummy image input
    if audio_input is None:
        audio_input = np.zeros((1, 1000, 1), dtype=np.float32)        # Dummy audio input
    if video_input is None:
        video_input = np.zeros((1, 225, 225, 3), dtype=np.float32)    # Dummy video input
    
    inputs = [text_input, image_input, audio_input, video_input]
    response = model.predict(inputs)  # Adjust this line based on your model's prediction method
    return response

# Function to handle video capture
def capture_video_frame():
    cap = cv2.VideoCapture(0)
    ret, frame = cap.read()
    cap.release()
    if ret:
        return frame
    else:
        return np.zeros((225, 225, 3), dtype=np.float32)  # Dummy video input

# Initialize the Tkinter root
root = tk.Tk()
root.title("Chatbot")

# Create the conversation display
conversation_frame = tk.Frame(root)
conversation_frame.pack(pady=10)

conversation_list = tk.Listbox(conversation_frame, width=80, height=20)
conversation_list.pack(side=tk.LEFT, fill=tk.BOTH)

scrollbar = tk.Scrollbar(conversation_frame, orient=tk.VERTICAL)
scrollbar.config(command=conversation_list.yview)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

conversation_list.config(yscrollcommand=scrollbar.set)

# Function to handle user input and generate response
def send_message(event=None):
    user_input = user_entry.get()
    if user_input:
        conversation_list.insert(tk.END, f"You: {user_input}")
        user_entry.delete(0, tk.END)
        
        # Process the input and generate response using the custom model
        processed_input = process_input_text(user_input)
        
        # Capture video frame
        video_frame = capture_video_frame()
        
        response_text = generate_response(trans_model, processed_input, video_input=video_frame)
        
        conversation_list.insert(tk.END, f"Bot: {response_text}")
        conversation_list.yview(tk.END)  # Scroll to the bottom

# Create the entry widget for user input
user_entry = tk.Entry(root, width=80)
user_entry.pack(pady=10)
user_entry.bind("<Return>", send_message)

# Create the send button
send_button = tk.Button(root, text="Send", command=send_message)
send_button.pack()

# Function to handle file upload
def upload_file():
    file_path = filedialog.askopenfilename()
    if file_path:
        conversation_list.insert(tk.END, f"File uploaded: {file_path}")
        # You can add further processing of the uploaded file here

# Create the file upload button
upload_button = tk.Button(root, text="Upload File", command=upload_file)
upload_button.pack()

# Function to handle microphone input
def record_audio():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        conversation_list.insert(tk.END, "Listening...")
        audio_data = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(audio_data)
            conversation_list.insert(tk.END, f"You (via microphone): {text}")
            user_entry.delete(0, tk.END)
            
            # Process the input and generate response using the custom model
            processed_input = process_input_text(text)
            
            # Capture video frame
            video_frame = capture_video_frame()
            
            response_text = generate_response(trans_model, processed_input, video_input=video_frame)
            
            conversation_list.insert(tk.END, f"Bot: {response_text}")
            conversation_list.yview(tk.END)  # Scroll to the bottom
        except sr.UnknownValueError:
            conversation_list.insert(tk.END, "Could not understand the audio")
        except sr.RequestError as e:
            conversation_list.insert(tk.END, f"Error with speech recognition service; {e}")

# Create the microphone button
microphone_button = tk.Button(root, text="Microphone", command=record_audio)
microphone_button.pack()

# Training loop to use files from ./knowledge_base
def training_loop(model, knowledge_base_dir):
    for filename in os.listdir(knowledge_base_dir):
        file_path = os.path.join(knowledge_base_dir, filename)
        if os.path.isfile(file_path):
            try:
               with open(file_path, 'r', encoding='utf-8') as file:
                    data = file.read()
                    processed_input = process_input_text(data)
                    # We will use dummy values for other inputs during training
            except UnicodeDecodeError:
                print(f"Error decoding file: {file_path}")
                processed_input = process_input_text(file_path)
                # We will use dummy values for other inputs during training
                image_input = np.zeros((1, 1920, 1080, 3), dtype=np.float32)
                audio_input = np.zeros((1, 1000, 1), dtype=np.float32)
                video_input = np.zeros((1, 225, 225, 3), dtype=np.float32)
                            
                # Train the model
                model.train_on_batch([processed_input, image_input, audio_input, video_input], processed_input)  # Adjust this line based on your model's training method

# Path to the knowledge base directory
knowledge_base_dir = 'C:/Users/jonny/Documents/PATH/ONI/knowledge_base/remembered_texts/'
# Start the training loop
#training_loop(trans_model, knowledge_base_dir)

# Run the Tkinter main loop
root.mainloop()

In [6]:
pip install speech_recognition

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement speech_recognition (from versions: none)
ERROR: No matching distribution found for speech_recognition


In [35]:
import requests

class Research():
    def __init__(self, api_key):
        self.api_key = api_key
        self.headers = {'Authorization': f'Bearer {self.api_key}'}

    def find_supporting_data(self, query):
        # Correcting the query parameter to 'q' as expected by the API
        url = f"https://serpapi.com/search?q={query}&api_key={self.api_key}"
        response = requests.get(url, headers=self.headers)
        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f"Failed to retrieve data, Status code: {response.status_code}, Message: {response.text}")

    def parse_data(self, data):
        parsed_data = []
        if data and 'results' in data:
            for item in data.get('results', []):
                parsed_data.append({
                    'title': item.get('title', 'No title provided'),
                    'summary': item.get('summary', 'No summary available'),
                    'url': item.get('url', 'No URL provided')
                })
        return parsed_data

# Usage
api_key = '5ca335203c1425c4161196327c12909e8a84a4f422adbef1fcf6d41d6c101f34'  # Replace with your actual API key
research_assistant = Research(api_key)
query = "happiness","health","fitness","philosophy","psychology","psychology research", "psychology theory", "psychology theory research","linear algebra","algebra","algebra theory","algebra theory research","geometry","geometry theory","geometry theory research", "Einstein","Maxwell","Fluid Dynamics","calculus","calculus theory","calculus theory research", "socrates","aristotle", "Daniel Khanmen"
try:
    extended_corpus = []
    results = research_assistant.find_supporting_data(query)
    parsed_results = research_assistant.parse_data(results)
    # Assuming 'extended_corpus' is defined elsewhere and accessible here
    extended_corpus.append([result['summary'] for result in parsed_results if result['summary'] not in extended_corpus])
except Exception as e:
    print(f"Error occurred: {e}")

print(results)


{'search_metadata': {'id': '665166c0c7ad4126276d8d9b', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/211e7922d0cc18a3/665166c0c7ad4126276d8d9b.json', 'created_at': '2024-05-25 04:19:12 UTC', 'processed_at': '2024-05-25 04:19:13 UTC', 'google_url': 'https://www.google.com/search?q=%28%27happiness%27%2C+%27health%27%2C+%27fitness%27%2C+%27philosophy%27%2C+%27psychology%27%2C+%27psychology+research%27%2C+%27psychology+theory%27%2C+%27psychology+theory+research%27%2C+%27linear+algebra%27%2C+%27algebra%27%2C+%27algebra+theory%27%2C+%27algebra+theory+research%27%2C+%27geometry%27%2C+%27geometry+theory%27%2C+%27geometry+theory+research%27%2C+%27Einstein%27%2C+%27Maxwell%27%2C+%27Fluid+Dynamics%27%2C+%27calculus%27%2C+%27calculus+theory%27%2C+%27calculus+theory+research%27%2C+%27socrates%27%2C+%27aristotle%27%2C+%27Daniel+Khanmen%27%29&oq=%28%27happiness%27%2C+%27health%27%2C+%27fitness%27%2C+%27philosophy%27%2C+%27psychology%27%2C+%27psychology+research%27%2C+%27psycholo

In [39]:
import numpy as np
import tensorflow as tf
import tkinter as tk
from tkinter import filedialog

import cv2
import os

# Placeholder functions for your custom model's text processing
def process_input_text(input_text):
    # Implement this to convert input_text to the format your model expects
    # For example, if the model expects tokenized input, tokenize the text here
    return input_text

def generate_response(model, text_input, image_input=None, audio_input=None, video_input=None):
    # Ensure that we provide four inputs to the model
    if image_input is None:
        image_input = np.zeros((1, 1920, 1080, 3), dtype=np.float32)  # Dummy image input
    if audio_input is None:
        audio_input = np.zeros((1, 1000, 1), dtype=np.float32)        # Dummy audio input
    if video_input is None:
        video_input = np.zeros((1, 225, 225, 3), dtype=np.float32)    # Dummy video input
    
    inputs = [text_input, image_input, audio_input, video_input]
    response = model.predict(inputs)  # Adjust this line based on your model's prediction method
    return response

# Function to handle video capture
def capture_video_frame():
    cap = cv2.VideoCapture(0)
    ret, frame = cap.read()
    cap.release()
    if ret:
        return frame
    else:
        return np.zeros((225, 225, 3), dtype=np.float32)  # Dummy video input

# Initialize the Tkinter root
root = tk.Tk()
root.title("Chatbot")

# Create the conversation display
conversation_frame = tk.Frame(root)
conversation_frame.pack(pady=10)

conversation_list = tk.Listbox(conversation_frame, width=80, height=20)
conversation_list.pack(side=tk.LEFT, fill=tk.BOTH)

scrollbar = tk.Scrollbar(conversation_frame, orient=tk.VERTICAL)
scrollbar.config(command=conversation_list.yview)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

conversation_list.config(yscrollcommand=scrollbar.set)

# Function to handle user input and generate response
def send_message(event=None):
    user_input = user_entry.get()
    if user_input:
        conversation_list.insert(tk.END, f"You: {user_input}")
        user_entry.delete(0, tk.END)
        
        # Process the input and generate response using the custom model
        processed_input = process_input_text(user_input)
        
        # Capture video frame
        video_frame = capture_video_frame()
        
        response_text = generate_response(trans_model, processed_input, video_input=video_frame)
        
        conversation_list.insert(tk.END, f"Bot: {response_text}")
        conversation_list.yview(tk.END)  # Scroll to the bottom

# Create the entry widget for user input
user_entry = tk.Entry(root, width=80)
user_entry.pack(pady=10)
user_entry.bind("<Return>", send_message)

# Create the send button
send_button = tk.Button(root, text="Send", command=send_message)
send_button.pack()

# Function to handle file upload
def upload_file():
    file_path = filedialog.askopenfilename()
    if file_path:
        conversation_list.insert(tk.END, f"File uploaded: {file_path}")
        # You can add further processing of the uploaded file here

# Create the file upload button
upload_button = tk.Button(root, text="Upload File", command=upload_file)
upload_button.pack()

# Function to handle microphone input
def record_audio():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        conversation_list.insert(tk.END, "Listening...")
        audio_data = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(audio_data)
            conversation_list.insert(tk.END, f"You (via microphone): {text}")
            user_entry.delete(0, tk.END)
            
            # Process the input and generate response using the custom model
            processed_input = process_input_text(text)
            
            # Capture video frame
            video_frame = capture_video_frame()
            
            response_text = generate_response(trans_model, processed_input, video_input=video_frame)
            
            conversation_list.insert(tk.END, f"Bot: {response_text}")
            conversation_list.yview(tk.END)  # Scroll to the bottom
        except sr.UnknownValueError:
            conversation_list.insert(tk.END, "Could not understand the audio")
        except sr.RequestError as e:
            conversation_list.insert(tk.END, f"Error with speech recognition service; {e}")

# Create the microphone button
microphone_button = tk.Button(root, text="Microphone", command=record_audio)
microphone_button.pack()


processed_input = process_input_text(results)
                    # We will use dummy values for other inputs during training



                # We will use dummy values for other inputs during training
image_input = np.zeros((1, 1920, 1080, 3), dtype=np.float32)
audio_input = np.zeros((1, 1000, 1), dtype=np.float32)
video_input = np.zeros((1, 225, 225, 3), dtype=np.float32)
                            
                # Train the model
trans_model.train_on_batch([processed_input, image_input, audio_input, video_input], processed_input)  # Adjust this line based on your model's training method

# Path to the knowledge base directory
knowledge_base_dir = 'C:/Users/jonny/Documents/PATH/ONI/knowledge_base/remembered_texts/'
# Start the training loop
training_loop(trans_model, knowledge_base_dir)

# Run the Tkinter main loop
root.mainloop()

ValueError: In a nested call() argument, you cannot mix tensors and non-tensors. Received invalid mixed argument: inputs=[{'search_metadata': {'id': '665166c0c7ad4126276d8d9b', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/211e7922d0cc18a3/665166c0c7ad4126276d8d9b.json', 'created_at': '2024-05-25 04:19:12 UTC', 'processed_at': '2024-05-25 04:19:13 UTC', 'google_url': 'https://www.google.com/search?q=%28%27happiness%27%2C+%27health%27%2C+%27fitness%27%2C+%27philosophy%27%2C+%27psychology%27%2C+%27psychology+research%27%2C+%27psychology+theory%27%2C+%27psychology+theory+research%27%2C+%27linear+algebra%27%2C+%27algebra%27%2C+%27algebra+theory%27%2C+%27algebra+theory+research%27%2C+%27geometry%27%2C+%27geometry+theory%27%2C+%27geometry+theory+research%27%2C+%27Einstein%27%2C+%27Maxwell%27%2C+%27Fluid+Dynamics%27%2C+%27calculus%27%2C+%27calculus+theory%27%2C+%27calculus+theory+research%27%2C+%27socrates%27%2C+%27aristotle%27%2C+%27Daniel+Khanmen%27%29&oq=%28%27happine